In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import networkx as nx
import matplotlib.pyplot as plt
import ast

In [ ]:
PARENT = "../Sweep/"
DATA_FOLDER = PARENT + "data_WESCO/"
DATA_FOLDER_CACHE = DATA_FOLDER + 'cache/'
T = 5

In [35]:
with open(DATA_FOLDER_CACHE + 'new_X.npy', 'rb') as f:
    X = np.load(f)

In [36]:
np.unique(X)

array([0.00000000e+00, 1.66666667e-02, 1.78571429e-02, ...,
       2.04000000e+02, 2.05000000e+02, 2.06000000e+02])

In [37]:
def buildAdjacencyMatrix(rows, cols):
    
    grid_graph = nx.grid_2d_graph(rows, cols)  ## connect all cells

    for u, v in grid_graph.edges():
        grid_graph[u][v]['weight'] = 1
    
    # for i in range(rows):
    #     for j in range(cols):
    #         if i > 0 and j > 0:
    #             grid_graph.add_edge((i, j), (i-1, j-1))  # top-left diagonal
    #         if i > 0 and j < cols - 1:
    #             grid_graph.add_edge((i, j), (i-1, j+1))  # top-right diagonal
    #         if i < rows - 1 and j > 0:
    #             grid_graph.add_edge((i, j), (i+1, j-1))  # bottom-left diagonal
    #         if i < rows - 1 and j < cols - 1:
    #             grid_graph.add_edge((i, j), (i+1, j+1))  # bottom-right diagonal

    def add_walls(grid_graph):

        with open(DATA_FOLDER_CACHE + 'walls.txt', 'r') as file:
            lines = file.readlines()
            filtered_lines = [l for l in lines if "#" not in l] 
            edges_to_remove = [ast.literal_eval(line.strip()) for line in filtered_lines]
        
        for n1, n2 in edges_to_remove:
            grid_graph.remove_edge(n1, n2)
            #grid_graph[n1][n2]['weight'] = 0.2

    add_walls(grid_graph)
    adj_matrix = nx.to_numpy_array(grid_graph)
    
    # G = nx.from_numpy_array(adj_matrix)
    # pos = {(i * cols + j): (j, -i) for i in range(rows) for j in range(cols)}
    # nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=300, font_size=8)
    # plt.show()
    
    # print(adj_matrix.shape)
    return adj_matrix

In [38]:
buildAdjacencyMatrix(8, 25)

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [39]:
def getTimeSeriesDataFromCell(tensor, i, j):
    cell = tensor[:, :, :, i, j] #get all samples, get all timesteps, get all features of cell i,j
    cell_t = np.transpose(cell, (0, 2, 1)) # result is an array shaped (sample, feature, time)
    return cell_t

def buildFeatureMatrix(tensor):  
    assert len(tensor.shape) == 5 
    samples, timesteps, features, rows, cols  = tensor.shape
    ## a feature matrix with size (number of nodes, number of features, timesteps)
    ## e.g fM[0,0] extracts the timeseries corresponding to feature 0 for node 0
    nodes = rows * cols
    featureMatrixes = np.empty((nodes, samples, features, timesteps))
    for i in range(rows):
        for j in range(cols):
            n = i * cols + j
            featureMatrixes[n] = getTimeSeriesDataFromCell(tensor, i, j)

    #print(featureMatrixes.shape)
    featureMatrixes = np.transpose(featureMatrixes, (1, 3, 0, 2))
    print(featureMatrixes.shape)
    return featureMatrixes        

In [40]:
X_no_loc = X[:, :, :, :, :] ## get rid of the location slice
adjMatrix = buildAdjacencyMatrix(8, 25)
featureMatrixes = buildFeatureMatrix(X_no_loc)  #(READINGS, 5, 200, 16)

samples, timesteps, features, rows, cols  = X_no_loc.shape

(4409, 5, 200, 16)


In [41]:
# Save adj to pickle file.
import pickle

adjMatrix = adjMatrix.astype(np.float32)

adj_matrix_file = DATA_FOLDER_CACHE + "/adj_mx.pkl"
with open(adj_matrix_file, 'wb') as f:
    pickle.dump([range(200), {i: i for i in range(200)}, adjMatrix], f, protocol=2)

featureMatrixes = featureMatrixes.astype(np.float32)


In [42]:
featureMatrixes.shape

(4409, 5, 200, 16)

In [43]:
with open(DATA_FOLDER_CACHE + 'new_y.npy', 'rb') as f:
    y = np.load(f)

In [44]:
y

array([[0.        , 1.23333333, 1.03333333, 1.03333333],
       [0.        , 1.        , 0.7       , 1.15      ],
       [0.        , 0.98333333, 0.76666667, 1.25      ],
       ...,
       [1.14285714, 2.2       , 3.05      , 5.875     ],
       [1.14285714, 2.2       , 3.11666667, 6.01666667],
       [1.14285714, 2.2       , 3.05      , 5.96666667]])

In [45]:
## recall notebook one: how where we defining y? 100, 101, 102, 103
"""
y[:, 0] represents readings of sensor 100
...
y[:, 3] represents readings of sensor 103
"""
y.shape

(4409, 4)

In [46]:
print(type(y), y.shape)

<class 'numpy.ndarray'> (4409, 4)


In [47]:
import torch

In [ ]:
sensor_indices = {"davinci": 24, "metcalfe": 78, "showroom": 155, "theater": 161}
 

reshaped_y = np.random.rand(samples, 1, rows * cols, 1)


# Fill in the values for the 4 sensors
for i, sensor_index in enumerate(sensor_indices.values()):
    reshaped_y[:, 0, sensor_index, 0] = y[:, i]

y = reshaped_y


In [49]:
y.shape

(4409, 1, 200, 1)

In [ ]:
fS = featureMatrixes.shape
yS = y.shape


sweep_filepath = DATA_FOLDER_CACHE + f"new_model_sweep.npz"


np.savez_compressed(
        sweep_filepath,
        x=featureMatrixes,
        y=y
)



In [51]:
print (y)

[[[[0.1891728 ]
   [0.75371179]
   [0.44078092]
   ...
   [0.91728714]
   [0.84057797]
   [0.29911774]]]


 [[[0.61685681]
   [0.6908601 ]
   [0.24803723]
   ...
   [0.27482839]
   [0.06420326]
   [0.20211824]]]


 [[[0.02461357]
   [0.26726842]
   [0.40391369]
   ...
   [0.35253896]
   [0.68537169]
   [0.39941173]]]


 ...


 [[[0.16491444]
   [0.59862441]
   [0.24236413]
   ...
   [0.51609529]
   [0.35495501]
   [0.98889733]]]


 [[[0.69905668]
   [0.22605495]
   [0.8182092 ]
   ...
   [0.23443584]
   [0.33207307]
   [0.56669505]]]


 [[[0.85354205]
   [0.19631969]
   [0.70448368]
   ...
   [0.37854603]
   [0.77459218]
   [0.16933337]]]]


In [52]:
print(y)


[[[[0.1891728 ]
   [0.75371179]
   [0.44078092]
   ...
   [0.91728714]
   [0.84057797]
   [0.29911774]]]


 [[[0.61685681]
   [0.6908601 ]
   [0.24803723]
   ...
   [0.27482839]
   [0.06420326]
   [0.20211824]]]


 [[[0.02461357]
   [0.26726842]
   [0.40391369]
   ...
   [0.35253896]
   [0.68537169]
   [0.39941173]]]


 ...


 [[[0.16491444]
   [0.59862441]
   [0.24236413]
   ...
   [0.51609529]
   [0.35495501]
   [0.98889733]]]


 [[[0.69905668]
   [0.22605495]
   [0.8182092 ]
   ...
   [0.23443584]
   [0.33207307]
   [0.56669505]]]


 [[[0.85354205]
   [0.19631969]
   [0.70448368]
   ...
   [0.37854603]
   [0.77459218]
   [0.16933337]]]]


In [53]:
X_train

array([[[[ 17.,   0.,  93., ...,  28.,   1.,   1.],
         [ 17.,   0.,  93., ...,  27.,   1.,   0.],
         [ 17.,   0.,  93., ...,  26.,   1.,   0.],
         ...,
         [ 17.,   0.,  93., ...,  26.,   1.,   1.],
         [ 17.,   0.,  93., ...,  27.,   1.,   0.],
         [ 17.,   0.,  93., ...,  28.,   1.,   1.]],

        [[  3.,  45.,  94., ...,  28.,   1.,   1.],
         [  3.,  45.,  94., ...,  27.,   1.,   0.],
         [  3.,  45.,  94., ...,  26.,   1.,   0.],
         ...,
         [  3.,  45.,  94., ...,  26.,   1.,   1.],
         [  3.,  45.,  94., ...,  27.,   1.,   0.],
         [  3.,  45.,  94., ...,  28.,   1.,   1.]],

        [[  4.,   0.,  94., ...,  28.,   1.,   1.],
         [  4.,   0.,  94., ...,  27.,   1.,   0.],
         [  4.,   0.,  94., ...,  26.,   1.,   0.],
         ...,
         [  4.,   0.,  94., ...,  26.,   1.,   1.],
         [  4.,   0.,  94., ...,  27.,   1.,   0.],
         [  4.,   0.,  94., ...,  28.,   1.,   1.]],

        [[  4., 

In [54]:
y_train

array([[[[0.1891728 ],
         [0.75371179],
         [0.44078092],
         ...,
         [0.91728714],
         [0.84057797],
         [0.29911774]]],


       [[[0.61685681],
         [0.6908601 ],
         [0.24803723],
         ...,
         [0.27482839],
         [0.06420326],
         [0.20211824]]],


       [[[0.02461357],
         [0.26726842],
         [0.40391369],
         ...,
         [0.35253896],
         [0.68537169],
         [0.39941173]]],


       ...,


       [[[0.62518643],
         [0.60583608],
         [0.44199101],
         ...,
         [0.39785744],
         [0.63458037],
         [0.27628439]]],


       [[[0.46723715],
         [0.89986301],
         [0.17561518],
         ...,
         [0.56812285],
         [0.59516551],
         [0.2504576 ]]],


       [[[0.04027226],
         [0.91904892],
         [0.56531024],
         ...,
         [0.15117749],
         [0.1361319 ],
         [0.04152281]]]])

In [55]:
featureMatrixes[:training_samples]


array([[[[ 17.,   0.,  93., ...,  28.,   1.,   1.],
         [ 17.,   0.,  93., ...,  27.,   1.,   0.],
         [ 17.,   0.,  93., ...,  26.,   1.,   0.],
         ...,
         [ 17.,   0.,  93., ...,  26.,   1.,   1.],
         [ 17.,   0.,  93., ...,  27.,   1.,   0.],
         [ 17.,   0.,  93., ...,  28.,   1.,   1.]],

        [[  3.,  45.,  94., ...,  28.,   1.,   1.],
         [  3.,  45.,  94., ...,  27.,   1.,   0.],
         [  3.,  45.,  94., ...,  26.,   1.,   0.],
         ...,
         [  3.,  45.,  94., ...,  26.,   1.,   1.],
         [  3.,  45.,  94., ...,  27.,   1.,   0.],
         [  3.,  45.,  94., ...,  28.,   1.,   1.]],

        [[  4.,   0.,  94., ...,  28.,   1.,   1.],
         [  4.,   0.,  94., ...,  27.,   1.,   0.],
         [  4.,   0.,  94., ...,  26.,   1.,   0.],
         ...,
         [  4.,   0.,  94., ...,  26.,   1.,   1.],
         [  4.,   0.,  94., ...,  27.,   1.,   0.],
         [  4.,   0.,  94., ...,  28.,   1.,   1.]],

        [[  4., 